# Harvard's Endowment

## HBS Case
### *The Harvard Management Company and Inflation-Indexed Bonds*

# 1. READING: HMC's Approach

### 1. 
There are thousands of individual risky assets in which HMC can invest.  Explain why MV optimization across 1,000 securities is infeasible.

### 2.
Rather than optimize across all securities directly, HMC runs a two-stage optimization.
1. They build asset class portfolios with each one optimized over the securities of the specific asset class.  
2. HMC combines the asset-class portfolios into one total optimized portfolio.

In order for the two-stage optimization to be a good approximation of the full MV-optimization on all assets, what must be true of the partition of securities into asset classes?

### 3.
Should TIPS form a new asset class or be grouped into one of the other 11 classes?

### 4. 
Why does HMC focus on real returns when analyzing its portfolio allocation? Is this just a matter of scaling, or does using real returns versus nominal returns potentially change the MV solution?

### 5.
The case discusses the fact that Harvard places bounds on the portfolio allocation rather than implementing whatever numbers come out of the MV optimization problem.

How might we adjust the stated optimization problem in the lecture notes to reflect the extra constraints Harvard is using in their bounded solutions given in Exhibits 5 and 6?

### 6. 
Exhibits 5 shows zero allocation to domestic equities and domestic bonds across the entire computed range of targeted returns, (5.75% to 7.25%). Conceptually, why is the constraint binding in all these cases? What would the unconstrained portfolio want to do with those allocations and why?

### 7.
Exhibit 6 changes the constraints, (tightening them in most cases.) How much deterioration do we see in the mean-variance tradeoff that Harvard achieved?

***

# 2 Mean-Variance Optimization

### Data
You will need the file in the github repo, `data/multi_asset_etf_data.xlsx`.
- The time-series data gives monthly returns for the 11 asset classes and a short-term Treasury-bill fund return, (`SHV`.)
- The case does not give time-series data, so this data has been compiled outside of the case, and it intends to represent the main asset classes under consideration via various ETFs. For details on the specific securities/indexes, check the “Info” tab of the data.

### Excess Returns
- We consider `SHV` as the risk-free asset.
- We are going to analyze the problem in terms of **excess** returns, where `SHV` has been subtracted from the other columns.
- The risk-free rate changes over time, but the assumption is that investors know it’s value one-period ahead of time. Thus, at any given point in time, it is a risk-free rate for the next period. (This is often discussed as the "bank account" or "money market account" in other settings.)

### Adjustment
For ease of analysis, drop `QAI` from the dataset. Analyze the remaining 10 assets.

### Not Considered
- These are nominal returns-they are not adjusted for inflation, and in our calculations we are not making any adjustment for inflation.
- The exhibit data that comes via Harvard with the case is unnecessary for our analysis.

### Format
In the questions below, **annualize the statistics** you report.
- Annualize the mean of monthly returns with a scaling of 12.
- Annualize the volatility of monthly returns with a scaling of $\sqrt{12}$
- Note that we are not scaling the raw timeseries data, just the statistics computed from it (mean, vol, Sharpe).

### 1. Summary Statistics
* Calculate and display the mean and volatility of each asset’s excess return. (Recall we use volatility to refer to standard deviation.)
* Which assets have the best and worst Sharpe ratios? 

Recall that the Sharpe Ratio is simply the ratio of the mean-to-volatility of excess returns:

$$\text{sharpe ratio of investment }i = \frac{\tilde{\mu}_i}{\sigma_i}$$

Be sure to annualize all three statss (mean, vol, Sharpe).


In [106]:
import pandas as pd
import numpy as np

file_path = '../data/multi_asset_etf_data.xlsx'
data = pd.read_excel(file_path, sheet_name='excess returns').set_index('Date')
data.drop('QAI', axis=1, inplace=True)
data.tail()

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,SPY,TIP
Date,,,,,,,,,,
2025-01-31,-0.002198,0.024463,0.018388,0.044877,0.010472,0.003033,0.015780,0.064862,0.023724,0.010570
2025-02-28,0.011376,-0.001250,0.008855,0.026915,0.007087,0.025382,0.035246,-0.042722,-0.015310,0.018957
2025-03-31,0.007510,0.019885,0.008497,-0.001004,-0.013701,0.000577,-0.026225,-0.063669,-0.058562,0.003954
2025-04-30,0.054708,-0.088766,-0.001615,0.033962,-0.001817,0.007573,-0.024503,-0.009856,-0.011659,-0.001729
2025-05-31,-0.007297,0.012373,0.037522,0.045208,0.014835,-0.015094,0.006375,0.041935,0.060147,-0.009017


In [112]:
def mean_n_vol(df):
    """Calculate and display the mean and volatility of each asset's excess return"""
    mu = df.mean(axis=0).rename('mean') * 12
    vol = df.std(axis=0).rename('vol') * np.sqrt(12)
    return pd.concat([mu, vol], axis=1)

def calc_sharpe(df, func, mute=False):
    df = func(df)
    df['sharpe'] = df['mean'] / df['vol']
    if not mute:
        print(f"The one has best sharpe is {df['sharpe'].idxmax()}, with sharpe={max(df['sharpe'])}")
        print(f"The one has worst sharpe is {df['sharpe'].idxmin()}, with sharpe={min(df['sharpe'])}")
    return df['sharpe']

In [113]:
mean_n_vol(data)

,mean,vol
BWX,-0.007716,0.082789
DBC,-0.005292,0.166553
EEM,0.029339,0.176164
EFA,0.061775,0.150903
HYG,0.041371,0.075928
IEF,0.016404,0.063442
IYR,0.074916,0.168675
PSP,0.092561,0.213370
SPY,0.128141,0.142839
TIP,0.020502,0.051115


In [114]:
calc_sharpe(data, mean_n_vol)

The one has best sharpe is SPY, with sharpe=0.897102626289479
The one has worst sharpe is BWX, with sharpe=-0.09320174565672586


BWX   -0.093202
DBC   -0.031774
EEM    0.166542
EFA    0.409372
HYG    0.544873
IEF    0.258569
IYR    0.444143
PSP    0.433804
SPY    0.897103
TIP    0.401091
Name: sharpe, dtype: float64

### 2. Descriptive Analysis
* Calculate the correlation matrix of the returns. Which pair has the highest correlation? And the lowest?
* How well have TIPS done in our sample? Have they outperformed domestic bonds? Foreign bonds?


In [115]:
cor = data.corr()
cor

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,SPY,TIP
BWX,1.000000,0.191116,0.621673,0.602820,0.602555,0.580891,0.552557,0.526692,0.439994,0.675151
DBC,0.191116,1.000000,0.511667,0.500922,0.461887,-0.300207,0.280518,0.453303,0.432162,0.109006
EEM,0.621673,0.511667,1.000000,0.819925,0.691167,0.026704,0.584063,0.750109,0.687751,0.378792
EFA,0.602820,0.500922,0.819925,1.000000,0.787191,0.042639,0.699292,0.895320,0.845863,0.394821
HYG,0.602555,0.461887,0.691167,0.787191,1.000000,0.187258,0.739356,0.812157,0.793518,0.538648
IEF,0.580891,-0.300207,0.026704,0.042639,0.187258,1.000000,0.316532,0.022436,0.000815,0.754102
IYR,0.552557,0.280518,0.584063,0.699292,0.739356,0.316532,1.000000,0.749836,0.754711,0.598742
PSP,0.526692,0.453303,0.750109,0.895320,0.812157,0.022436,0.749836,1.000000,0.891687,0.408005
SPY,0.439994,0.432162,0.687751,0.845863,0.793518,0.000815,0.754711,0.891687,1.000000,0.381625
TIP,0.675151,0.109006,0.378792,0.394821,0.538648,0.754102,0.598742,0.408005,0.381625,1.000000


In [117]:
np.fill_diagonal(cor.values, -1)
idx_name, col_name = cor.stack().idxmax()
print(f"The highest correlation is {cor.values.max()}, with pair {idx_name}-{col_name}")

The highest correlation is 0.8953201243752301, with pair EFA-PSP


### 3. The MV frontier.
* Compute and display the weights of the tangency portfolios: $\text{w}^{\text{tan}}$.
* Does the ranking of weights align with the ranking of Sharpe ratios?
* Compute the mean, volatility, and Sharpe ratio for the tangency portfolio corresponding to
$\text{w}^{\text{tan}}$.


In [118]:
def wtan(cor, mu):
    """
    cor: (DataFrame) correlation matrix
    mu: (Series) average excess return
    """
    w = np.linalg.inv(cor.values) @ mu.values
    w_scaled = w / np.sum(w)
    return pd.Series(w_scaled, index=mu.index)

In [120]:
# weights
mu = mean_n_vol(data)["mean"] * 12
wtan(cor, mu)

BWX    0.617903
DBC   -0.682175
EEM   -0.244584
EFA   -0.283814
HYG   -0.046333
IEF    1.237577
IYR    0.167005
PSP   -0.302738
SPY   -0.343594
TIP    0.880754
dtype: float64

Compare rankings

In [121]:
sharpe = calc_sharpe(data, mean_n_vol, mute=True)
sharpe.sort_values().index.values

array(['BWX', 'DBC', 'EEM', 'IEF', 'TIP', 'EFA', 'PSP', 'IYR', 'HYG',
       'SPY'], dtype=object)

In [122]:
weights = wtan(cor, mu)
weights.sort_values().index.values

array(['DBC', 'SPY', 'PSP', 'EFA', 'EEM', 'HYG', 'IYR', 'BWX', 'TIP',
       'IEF'], dtype=object)

No, the weights do not align.

In [123]:
portfolio_return = (data.values * weights.values.reshape(1, -1)).sum(axis=1)  # excess return
mu_p = portfolio_return.mean() * 12
vol_p = portfolio_return.std() * np.sqrt(12)
sharpe_p = mu_p / vol_p
print(f"Mean: {mu_p}, volatility: {vol_p}, sharpe: {sharpe_p}")

Mean: -0.04896348042917821, volatility: 0.2681869773097092, sharpe: -0.18257217751715785



### 4. TIPS
Assess how much the tangency portfolio (and performance) change if...
* TIPS are dropped completely from the investment set.
* The expected excess return to TIPS is adjusted to be 0.0012 higher than what the historic sample shows.

Based on the analysis, do TIPS seem to expand the investment opportunity set, implying that Harvard should consider them as a separate asset?

***

# 3. Allocations


* Continue with the same data file as the previous section.

* Suppose the investor has a targeted mean excess return (per month) of $\tilde{\mu}^{\text{port}}$ = 0.01.

Build the following portfolios:

#### Equally-weighted (EW)
Rescale the entire weighting vector to have target mean $\tilde{\mu}^{\text{port}}$. Thus, the $i$ element of the weight vector is,

$$w^{\text{EW}}_i = \frac{1}{n}$$

#### “Risk-parity” (RP)
Risk-parity is a term used in a variety of ways, but here we have in mind setting the weight of the portfolio to be proportional to the inverse of its full-sample variance estimate. Thus, the $i$ element of the weight vector is,

$$w^{\text{RP}}_i = \frac{1}{\sigma_i^2}$$

#### Mean-Variance (MV)
As described in `Section 2`.

### Comparing

In order to compare all these allocation methods, rescale each weight vector, such that it has targeted mean return of $\tilde{\mu}^{\text{port}}$.

* Calculate the performance of each of these portfolios over the sample.
* Report their mean, volatility, and Sharpe ratio. 
* How does performance compare across allocation methods?

***

# 4. EXTRA: Out-of-Sample Performance

### 1. One-step Out-of-Sample (OOS) Performance
Let’s divide the sample to both compute a portfolio and then check its performance out of sample.
* Using only data through the end of `2023`, compute the weights built in Section 3.
* Rescale the weights, (using just the in-sample data,) to set each allocation to have the same mean return of $\tilde{\mu}^{\text{port}}$.
* Using those weights, calculate the portfolio’s Sharpe ratio within that sample.
* Again using those weights, (derived using data through `2023`,) calculate the portfolio’s OOS Sharpe ratio, which is based only on performance in `2024-2025`.

### 2. Rolling OOS Performance

Iterate the Out-of-Sample performance every year, not just the final year. Namely,
* Start at the end of `2015`, and calculate the weights through that time. Rescale them using the mean returns through that time.
* Apply the weights to the returns in the upcoming year, (`2016`.)
* Step forward a year in time, and recompute.
* Continue until again calculating the weights through `2023` and applying them to the returns in `2024-2025`.

Report the mean, volatility, and Sharpe from this dynamic approach for the following portfolios:
* mean-variance (tangency)
* equally-weighted
* risk-parity
* regularized

***

# 5. EXTRA: Without a Riskless Asset

Re-do Section 2 above, but in the model without a risk-free rate.

That is, build the MV allocation using the two-part formula in the `Mean-Variance` section of the notes.
* This essentially substitutes the risk-free rate with the minimum-variance portfolio.
* Now, the allocation depends nonlinearly on the target mean return, $\tilde{\mu}^{\text{port}}$. (With a risk-free rate, we simply scale the weights up and down to achieve the mean return.)

You will find that, conceptually, the answers are very similar.

***

# 6. EXTRA: Bayesian Allocation

Add the following allocation among the choices in `Section 3`...

#### Regularized (REG)
Much like the Mean-Variance portfolio, set the weights proportional to 

$$w^{\text{REG}} \sim \widehat{\Sigma}^{-1}\mux$$

but this time, use a regularized covariance matrix,

$$\widehat{\Sigma} = \frac{\Sigma + \Sigma_D}{2}$$

where $\Sigma_D$ denotes a *diagonal* matrix of the security variances, with zeros in the off-diagonals.

Thus, $\widehat{\Sigma}$ is obtained from the usual covariance matrix, $\Sigma$, but shrinking all the covariances to half their estimated values.

# 7. EXTRA: Inefficient Tangency

Return to analyzing the excess returns, as in `Section 3`. Include the hedge-fund ETF, QAI.

You might find that the tangency portfolio has a negative mean return. 
- It is on the inefficient portion of the MV frontier.

Calculate the optimal allocation by shorting the tangency portfolio. (That is, multiply the tangency weights by negative one to determine the allocation weights.)

Re-do the analysis of Section 3. Does the addition of `QAI` change much.